### Imports

In [139]:
### Standard imports
import os
import shutil
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from glob import glob
from os.path import join
from conv4d import conv4d


### Custom helpers
import sys
from helpers import *

### Grey masks

In [2]:
masks = glob("/Volumes/hd_4tb/masks/plip/*")
masks = [{
    "code": os.path.basename(mask).split("_")[0],
    "data": nib.load(mask).get_data(),
} for mask in masks]

def _in_mask(masks, x, y, z):
    result = dict()
    for mask in masks:
        code = mask["code"]
        data = mask["data"]
        result[code] = bool(data[x, y, z])
    return result

print(_in_mask(masks, 49, 66, 33))

{'001716': True, '013136': False, '073720': False, '125909': False, '131520': False, '156999': False, '176064': False, '182355': False, '216192': False, '275836': False, '291082': False, '414207': False, '426426': False, '447647': False, '450767': False, '477522': False, '525931': False, '532911': False, '533294': False, '534482': False, '707656': False, '752691': False, '779062': False, '791103': False, '831650': False, '877949': False, '911981': False, '954861': False}


### fMRI

In [104]:
def _load_volume(fmri, x, y, z, t):
    volume = nii_input(fmri[:, :, :, t], x, y, z)
    return np.array(volume)

def _load(filepath):
    return nib.as_closest_canonical(nib.load(filepath))

def _get_data(filepath):
    image = _load(filepath)
    return image.get_data()

def _time_map(session):
    _map = {
        "s1": "000", "s2": "2MO", "s3": "6MO", "s4": "12MO", "s5": "24MO"
    }
    return _map[session]

def _get(row, item):
    return row[item]

def _fmri_path(row):
    _input = "/Volumes/hd_4tb/raw"
    project = _get_project(row)
    subject = _get(row, "subject")
    time_session = _get(row, "time_session")
    task = _get(row, "task")
    return join(_input, project, _time_map(time_session), subject, task)

def nii_input(data, x, y, z, r = 2):
    return data[
        x - r : x + r + 1,
        y - r : y + r + 1,
        z - r : z + r + 1
    ]

### Onsets

In [74]:
def _onset_time(onsets, curr_time):
    onset = onsets[onsets["ons"] < curr_time]
    onset = onset.sort_values("ons", ascending=False)
    if len(onset) > 0:
        _time = onset.iloc[0].ons
        return curr_time - _time
    return False

def _stim_time(df, stimuli, curr_time):
    stim = df[df.category == stimuli]
    return _onset_time(stim, curr_time)

def _keypress_times(df, button, curr_time):
    keys = df[(
        (df["category"] == "keypress") &
        (df["stimulus"] == button)
    )]
    return _onset_time(keys, curr_time)

def last_onset(onset_df, task, curr_time, max_time=1000):
    task_stimuli = {
        "gonogo":       ["Go", "NoGo"],
        "conscious":    ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad"],
        "nonconscious": ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad"],
        "workingmemSB": ["Baseline", "NonTarget", "Target"],
        "workingmemMB": ["Baseline", "NonTarget", "Target"],
    }
    all_stimuli  = set(np.concatenate(list(task_stimuli.values())))
    onset_timing = {stimuli: max_time for stimuli in all_stimuli}
    for button in ["1", "6"]:
        onset_timing[button] = _keypress_times(onset_df, button, curr_time)
        
    for stimuli in task_stimuli[task]:
        _time = _stim_time(onset_df, stimuli, curr_time)
        if _time:
            onset_timing[stimuli] = _time
    print(onset_timing)

### Model

In [142]:
def _checkpoint():
    return False

def _load_model():
    next_volume = keras.Input(shape=(5, 5, 5,), name="next_volume")
    prev_volume = keras.Input(shape=(5, 5, 5,), name="prev_volume")

    prev_volume = layers.Conv3D()(prev_volume)
#     prev_volume = keras.Input(shape=(2, 5, 5, 5,), name="prev_volume")
#     info        = keras.Input(shape=(42,), name="info")
    
# #     title_features = layers.Embedding(num_words, 64)(title_input)
# #     body_features = layers.Embedding(num_words, 64)(body_input)

#     x = layers.concatenate([prev_volume, next_volume, info])

#     bold_signal = layers.Dense(1, activation="sigmoid", name="bold_signal")(x)

#     model = keras.Model(inputs=[prev_volume, next_volume, info], outputs=[bold_signal])
#     model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
#           loss={"bold_signal": 'binary_crossentropy'},
#           loss_weights=[1.])
#     if _checkpoint():
#         model.load_weights("checkpoint_path")
    return model

In [143]:
_load_model()

TypeError: data type not understood

### Train

For working with multiple inputs/outputs see [here](https://www.tensorflow.org/guide/keras/functional#models_with_multiple_inputs_and_outputs)

In [135]:
from tqdm import tqdm_notebook as tqdm

df                = pd.read_csv("/Volumes/hd_4tb/project/model_input.csv")
available_volumes = np.load("available_volumes.npy")
train_cols        = [c for c in df.columns if c.startswith("is_")] + ["age"]
model = _load_model()
batch_size = 64

for i, row in tqdm(df.iterrows()):
    TR = 2 if _get(row, "is_mb") == 0 else 0.71
    fmri_path = _fmri_path(row)
    fmri      = _get_data(join(fmri_path, "normalized.nii.gz"))
    grey      = _get_data(join(fmri_path, "..", "structural", "gm_probseg.nii.gz"))
    onset_df  = pd.read_csv(join(fmri_path, "onsets.csv"))
    info      = row[train_colsa]
    
    batch = {"prev_volume": list(), "next_volume": list()}
    bold_sigal = list()
    for t in range(1, fmri.shape[3]):
        onsets = last_onset(onset_df, _get(row, "task"), TR * t, max_time=1000)
        for x, y, z in tqdm(available_volumes):
            if len(batch["prev_volume"]) == batch_size:
                batch["info"] = pd.concat([
                    pd.concat([onsets] * batch_size),
                    pd.concat([info] * batch_size)
                ], axis=1)
                print(batch["info"].shape)
                model.fit(
                    batch, {'priority': bold_signal},
                    epochs=2, batch_size=32
                )
                batch = {"prev_volume": list(), "next_volume": list()}
                bold_signal = list()
            
            grey_data = nii_input(grey, x, y, z)
            batch["prev_volume"].append(np.multiply(grey_data, _load_volume(fmri, x, y, z, t-1)))
            batch["next_volume"].append(np.multiply(grey_data, _load_volume(fmri, x, y, z, t+1)))
            bold_signal.append(fmri[x, y, z, t])

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 2, 5, 5, 5), (None, 2, 5, 5, 5), (None, 29), (None, 13)]